In [1]:
%%capture
!pip install git+https://github.com/Nixtla/neuralforecast.git
!pip install git+https://github.com/Nixtla/hierarchicalforecast.git
!pip install datasetsforecast 
!pip install hyperopt
!git clone https://github.com/dluuo/HINT.git

In [2]:
import numpy as np
import pandas as pd

from neuralforecast.utils import augment_calendar_df

from hierarchicalforecast.evaluation import scaled_crps, msse
from datasetsforecast.hierarchical import HierarchicalData, HierarchicalInfo
from neuralforecast import NeuralForecast
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.models import (
    MLP, NBEATS, NBEATSx, NHITS, 
    RNN, GRU, LSTM, DilatedRNN, TCN,
    TFT, VanillaTransformer, Informer, Autoformer, PatchTST,
    HINT,
)
from neuralforecast.auto import (
    AutoHINT,
    AutoNHITS,
)
from neuralforecast.losses.pytorch import GMM, sCRPS
from ray import tune

#import wrangled datasets
from HINT.src.data.data import HierarchicalDataset

In [3]:
def evaluate_forecast(nf, hdataset, Y_train_df, Y_test_df, hier_idxs, hier_levels, n_samples):
    # Extract hint.model from nf core
    model_name = type(nf.models[0]).__name__
    model = nf.models[0].model

    # Parse Yq_hat_sample and Y_test
    # Keep only quantile columns from Y_hat_df
    # Removing mean and median default outputs
    Y_hat_df_list = [nf.predict() for _ in range(n_samples)]
    model_columns = [model_name + n for n in model.loss.output_names]

    # Parse shapes
    n_series = len(Y_test_df.unique_id.unique())
    horizon = len(Y_test_df.ds.unique())   
    n_quantiles = len(model_columns)-1
    
    Y_hat_sample = []
    for sample_idx in range(n_samples):
        Y_hat = Y_hat_df_list[sample_idx][model_columns].values
        Y_hat_sample.append(Y_hat.reshape(1, n_series, horizon, -1))
    Y_hat_sample = np.concatenate(Y_hat_sample, axis=0)
    Ymean_hat_sample = Y_hat_sample[:,:,:,0] # Default mean
    Yq_hat_sample = Y_hat_sample[:,:,:,3:] # Drop mean/median default
    Y_test = Y_test_df['y'].values.reshape(n_series, horizon)
    Y_train = Y_train_df['y'].values.reshape(n_series, -1)

    # Compute bootstraped sCRPS, mean and 95% confidence intervals
    sample_scrps = []
    sample_relmse = []
    n_samples = Yq_hat_sample.shape[0]
    for sample_idx in range(n_samples):
        relmse = hdataset._get_hierarchical_msse(hier_idxs=hier_idxs,
                                          Y=Y_test,
                                          Y_hat=Ymean_hat_sample[sample_idx],
                                          Y_train=Y_train)
        sample_relmse.append(relmse)

        scrps = hdataset._get_hierarchical_scrps(hier_idxs=hier_idxs,
                                        Y=Y_test,
                                        Yq_hat=Yq_hat_sample[sample_idx],
                                        q_to_pred=quantiles)
        sample_scrps.append(scrps)

    mean_relmse = np.mean(np.array(sample_relmse), axis=0)
    std_relmse  = np.std(np.array(sample_relmse), axis=0)
    relmse_results = [f'{mean_relmse[level_idx]:.4f}±{(1.96 * std_relmse[level_idx]):.4f}' \
                     for level_idx in range(len(mean_relmse))]

    mean_scrps = np.mean(np.array(sample_scrps), axis=0)
    std_scrps  = np.std(np.array(sample_scrps), axis=0)
    scrps_results = [f'{mean_scrps[level_idx]:.4f}±{(1.96 * std_scrps[level_idx]):.4f}' \
                     for level_idx in range(len(mean_scrps))]
    
    evaluation_df = dict(Levels=hier_levels,
                         sCRPS=scrps_results,
                         relMSE=relmse_results)
    evaluation_df = pd.DataFrame(evaluation_df)
    return evaluation_df

In [4]:
dataset = 'TourismSmall'

#------------------------ Read/Parse Data ------------------------#
hdataset = HierarchicalDataset()
data = hdataset.load_process_data(dataset=dataset)

# Experiment parameters
horizon = data['seasonality']
seasonality = data['seasonality']
hier_idxs = data['hier_idxs']
hier_levels = data['hier_levels']
freq = data['freq']
Y_df = data['Y_df']
S_df = data['S_df']
level = np.arange(0, 100, 2)
qs = [[50-lv/2, 50+lv/2] for lv in level]
quantiles = np.sort(np.concatenate(qs)/100)
n_series = len(S_df)

# Train/Test split
Y_test_df  = Y_df.groupby('unique_id').tail(horizon)
Y_train_df = Y_df.drop(Y_test_df.index)


100%|██████████| 1.30M/1.30M [00:00<00:00, 2.57MiB/s]


In [5]:
%%capture
#----------------------- Fit/Predict HINT ------------------------#
# HINT := BaseNetwork + Distribution + Reconciliation
config = {
            "scaler_type": tune.choice(['robust']),
            "futr_exog_list": tune.choice([[f'y_[lag{seasonality}]']]),
            "learning_rate": tune.choice([1e-3, 5e-4, 1e-4]),                                         # Initial Learning rate
            "max_steps": tune.choice([500, 1000, 1500, 2000, 2500, 3000]),                            # Number of SGD steps
            "input_size": tune.choice([2*seasonality, 3*seasonality, 4*seasonality]),                 # input_size = multiplier * horizon
            "valid_batch_size": tune.choice([n_series]), 
            "val_check_steps": tune.choice([100]),                                                    # Compute validation every 100 steps
            "early_stop_patience_steps": tune.choice([3]),
            "num_lr_decays": tune.choice([-1, 3]),
            "random_seed": tune.randint(1, 5), 
            "reconciliation": tune.choice(['BottomUp', 'MinTraceOLS', 'MinTraceWLS']),
            # "inference_input_size": tune.choice([2*seasonality, 3*seasonality, 4*seasonality]),
        }

model = AutoHINT(h=horizon, S=S_df.values,
                 cls_model=NHITS,
                 config=config,
                 loss=GMM(n_components=18, quantiles=quantiles),
                 valid_loss=sCRPS(quantiles=quantiles),
                 num_samples=20)

nf = NeuralForecast(models=[model], freq=freq)
nf.fit(df=Y_train_df, val_size=horizon)

INFO:lightning_fabric.utilities.seed:Global seed set to 2


In [6]:
evaluation_df = evaluate_forecast(nf=nf, hdataset=hdataset,
                                Y_test_df=Y_test_df, Y_train_df=Y_train_df,
                                hier_idxs=hier_idxs, hier_levels=hier_levels,
                                n_samples=8)
evaluation_df

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

,Levels,sCRPS,relMSE
0,Overall,0.0685±0.0002,0.0931±0.0011
1,Country,0.0307±0.0004,0.0642±0.0010
2,Country/Purpose,0.0561±0.0004,0.0974±0.0020
3,Country/Purpose/State,0.0868±0.0002,0.1479±0.0020
4,Country/Purpose/State/CityNonCity,0.1004±0.0006,0.1375±0.0015
